In [2]:
import tensorflow as tf
import os
import numpy as np


In [3]:
def parse_data(filename, label) :
    img_read = tf.io.read_file(filename)
    img_decode = tf.image.decode_jpeg(img_read, channels=3)
#     img = tf.image.resize(img_decode, [227,227])
    return img_decode, label

def create_dataset(filenames, labels, batch_size) :
    dataset = tf.data.Dataset.from_tensor_slices((filenames,labels))
    dataset = dataset.map(parse_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=len(filenames))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    return dataset

def read_labels(path) :
    img_classes = os.listdir(path)
    total_len = len(os.listdir(path+img_classes[0])) + len(os.listdir(path+img_classes[1])) + \
    len(os.listdir(path+img_classes[2]))+ len(os.listdir(path+img_classes[3])) + \
    len(os.listdir(path+img_classes[4]))+ len(os.listdir(path+img_classes[5]))+\
    len(os.listdir(path+img_classes[6]))

    names = []
    labels = np.zeros((total_len, len(img_classes)))
    j=0
    for i, img_class in enumerate(img_classes) :
        for img in os.listdir(os.path.join(path, img_class)) :
            names.append(os.path.join(path, img_class, img))
            labels[j][i] = 1
            j+=1
        
    return names, labels


batch_size = 16
images, labels = read_labels('./train/train/')
dataset = create_dataset(images, labels, batch_size)

In [4]:
# train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True,
# )

# test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
#     rescale=1./255)

# train_dataset = train_gen.flow_from_directory(
#     './train/train/',
#     target_size=(227,227),
#     batch_size=8,
#     class_mode='categorical'
# )

# test_dataset = test_gen.flow_from_directory(
#     './test/test/',
#     target_size=(227,227),
#     batch_size=8,
#     class_mode='categorical'
# )

In [5]:
from keras.layers import Conv2D, Add, BatchNormalization, Activation, Input, ZeroPadding2D, MaxPool2D, GlobalAvgPool2D, Dense
from keras import Model
input = Input(shape=(227,227,3), dtype='float32', name='input')

In [6]:
def conv1_layer(x) :
    x = ZeroPadding2D(padding=(3,3))(x)
    x = Conv2D(64, kernel_size=(7,7), strides=(2,2), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(padding=(1,1))(x)
    
    return x

In [7]:
def conv2_layer(x) :
    x = MaxPool2D((3,3), strides=(2,2))(x)
    short_cut = x 
    
    for i in range(3) :
        if i == 0 :
            x = Conv2D(64, kernel_size=(1,1), strides=(1,1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(64, kernel_size=(3, 3), strides=(1,1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(256, kernel_size=(1,1), strides=(1,1), padding='valid')(x)
            x = BatchNormalization()(x)
            
            short_cut = Conv2D(256, kernel_size=(1,1), strides=(1,1), padding='valid')(short_cut)
            short_cut = BatchNormalization()(short_cut)
            
            x = Add()([x, short_cut])
            
            x = Activation('relu')(x)
            
            short_cut = x
        
        else :
            
            x = Conv2D(64, kernel_size=(1,1), strides=(1,1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(64, kernel_size=(3, 3), strides=(1,1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(256, kernel_size=(1,1), strides=(1,1), padding='valid')(x)
            x = BatchNormalization()(x)
            
            x = Add()([x, short_cut])
            
            x = Activation('relu')(x)
            
            short_cut = x
                        
    return x

In [8]:
def conv3_layer(x) :
    short_cut = x
    
    for i in range(4) :
        if i == 0 :
            x = Conv2D(128, kernel_size=(1,1), strides=(2,2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(512, kernel_size=(1,1), strides=(1,1), padding='valid')(x)
            x = BatchNormalization()(x)
            
            short_cut = Conv2D(512, kernel_size=(1,1), strides=(2, 2), padding='valid')(short_cut)
            short_cut = BatchNormalization()(short_cut)

            x = Add()([x, short_cut])
            
            x = Activation('relu')(x)
            
            short_cut = x
            
        else :
            x = Conv2D(128, kernel_size=(1,1), strides=(1,1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(512, kernel_size=(1,1), strides=(1,1), padding='valid')(x)
            x = BatchNormalization()(x)
            
            x = Add()([x, short_cut])
            
            x = Activation('relu')(x)
            
            short_cut = x
            
    return x

In [9]:
def conv4_layer(x) :
    short_cut = x
    
    for i in range(6) :
        if i == 0 :
            x = Conv2D(256, kernel_size=(1,1), strides=(2,2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(256, kernel_size=(3,3), strides=(1,1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(1024, kernel_size=(1,1), strides=(1,1), padding='valid')(x)
            x = BatchNormalization()(x)
            
            short_cut = Conv2D(1024, kernel_size=(1,1), strides=(2,2), padding='valid')(short_cut)
            short_cut = BatchNormalization()(short_cut)
            
            x = Add()([x, short_cut])
            
            x = Activation('relu')(x)
            
            short_cut = x
            
        else : 
            x = Conv2D(256, kernel_size=(1,1), strides=(1,1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(256, kernel_size=(3,3), strides=(1,1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(1024, kernel_size=(1,1), strides=(1,1), padding='valid')(x)
            x = BatchNormalization()(x)
            
            x = Add()([x, short_cut])
            
            x = Activation('relu')(x)
            
            short_cut = x
    
    return x
            

In [10]:
def conv5_layer(x) :
    short_cut = x
    
    for i in range(3) :
        if i == 0 :
            x = Conv2D(512, kernel_size=(1,1), strides=(2,2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(512, kernel_size=(3,3), strides=(1,1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(2058, kernel_size=(1,1), strides=(1,1), padding='valid')(x)
            x = BatchNormalization()(x)
            
            short_cut = Conv2D(2058, kernel_size=(1,1), strides=(2,2), padding='valid')(short_cut)
            short_cut = BatchNormalization()(short_cut)
            
            x = Add()([x, short_cut])
            
            x = Activation('relu')(x)
            
        else :
            x = Conv2D(512, kernel_size=(1,1), strides=(1,1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(512, kernel_size=(3,3), strides=(1,1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(2058, kernel_size=(1,1), strides=(1,1), padding='valid')(x)
            x = BatchNormalization()(x)
            
            x = Add()([x, short_cut])
            
            x = Activation('relu')(x)
            
            short_cut = x
            
    return x

In [11]:
x = conv1_layer(input)
x = conv2_layer(x)
x = conv3_layer(x)
x = conv4_layer(x)
x = conv5_layer(x)

x = GlobalAvgPool2D()(x)
output = Dense(7, activation='softmax')(x)

model = Model(input, output)

model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 233, 233, 3)  0           input[0][0]                      
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 114, 114, 64) 9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 114, 114, 64) 256         conv2d[0][0]                     
_______________________________________________________________________________________

In [12]:
metrics = [
    tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='AUC')
]

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=5),
    tf.keras.callbacks.ModelCheckpoint('./check/cp.ckpt',
                                        save_weights_only=True,
                                      save_freq='epoch'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy',
                                        factor=0.1, patience=5,
                                        min_lr=1e-13)
]

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
             loss=tf.keras.losses.CategoricalCrossentropy(),
             metrics=metrics)

In [14]:
history = model.fit(dataset,
                    batch_size=16,
                    epochs=3, 
                    callbacks=callbacks)

Epoch 1/3
107/107 [==============================] - 892s 8s/step - loss: 2.1398 - accuracy: 0.2750 - precision: 0.3557 - recall: 0.0718 - AUC: 0.6462
Epoch 2/3
107/107 [==============================] - 866s 8s/step - loss: 1.7624 - accuracy: 0.3451 - precision: 0.4890 - recall: 0.0919 - AUC: 0.7148
Epoch 3/3
107/107 [==============================] - 869s 8s/step - loss: 1.7027 - accuracy: 0.3716 - precision: 0.5230 - recall: 0.1137 - AUC: 0.7341


In [17]:
import cv2
img = cv2.imread('./train/train/guitar/pic_004.jpg')
# img = cv2.imread('./train/train/person/pic_004.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, dsize=(227,227))
img = tf.reshape(img, (-1, img.shape[0], img.shape[1], img.shape[2]))
a = model.predict(img)
print(a)

[[0.13000453 0.13450815 0.18972123 0.07716474 0.05512882 0.27821234
  0.13526016]]


In [34]:
from keras import Sequential
from keras import layers

resnet50 = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', input_shape=(227,227,3))

for i, layer in enumerate(resnet50.layers) :
    if i < 143 :
        layer.trainable = False

resnet50.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 233, 233, 3)  0           input_15[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 114, 114, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 114, 114, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [35]:
model = Sequential()
model.add(resnet50)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(7, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 8, 8, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 14343     
Total params: 23,602,055
Trainable params: 14,990,343
Non-trainable params: 8,611,712
_________________________________________________________________


In [36]:
metrics = [
    tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='AUC')
]

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=5),
    tf.keras.callbacks.ModelCheckpoint('./check/cp.ckpt',
                                        save_weights_only=True,
                                      save_freq='epoch'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy',
                                        factor=0.1, patience=5,
                                        min_lr=1e-13)
]
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
             loss=tf.keras.losses.CategoricalCrossentropy(),
             metrics=metrics)


In [37]:
history = model.fit(dataset,
                    batch_size=16,
                    epochs=3, 
                    callbacks=callbacks)

Epoch 1/3
107/107 [==============================] - 200s 2s/step - loss: 0.7838 - accuracy: 0.7780 - precision: 0.8505 - recall: 0.6938 - AUC: 0.9527
Epoch 2/3
107/107 [==============================] - 220s 2s/step - loss: 0.2190 - accuracy: 0.9364 - precision: 0.9567 - recall: 0.9099 - AUC: 0.9947
Epoch 3/3
107/107 [==============================] - 229s 2s/step - loss: 0.0787 - accuracy: 0.9782 - precision: 0.9820 - recall: 0.9664 - AUC: 0.9996


In [39]:
import cv2
img = cv2.imread('./train/train/guitar/pic_004.jpg')
# img = cv2.imread('./train/train/person/pic_004.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, dsize=(227,227))
img = tf.reshape(img, (-1, img.shape[0], img.shape[1], img.shape[2]))
a = model.predict(img)
print(a[0])

[5.5802428e-07 7.1505241e-07 3.8401000e-04 9.9941790e-01 1.8549332e-04
 3.0917704e-07 1.0928332e-05]


In [40]:
from keras import Sequential
from keras import layers

resnet50 = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', input_shape=(227,227,3))

for i, layer in enumerate(resnet50.layers) :
    if i < 143 :
        layer.trainable = False

resnet50.summary()


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 233, 233, 3)  0           input_16[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 114, 114, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 114, 114, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [41]:
model = Sequential()
model.add(resnet50)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(7, activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 8, 8, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_4 ( (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 7)                 14343     
Total params: 23,602,055
Trainable params: 14,990,343
Non-trainable params: 8,611,712
_________________________________________________________________


In [42]:
metrics = [
    tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='AUC')
]

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=5),
    tf.keras.callbacks.ModelCheckpoint('./check/cp.ckpt',
                                        save_weights_only=True,
                                      save_freq='epoch'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy',
                                        factor=0.1, patience=5,
                                        min_lr=1e-13)
]
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
             loss=tf.keras.losses.CategoricalCrossentropy(),
             metrics=metrics)


In [43]:
history = model.fit(dataset,
                    batch_size=16,
                    epochs=4, 
                    callbacks=callbacks)

Epoch 1/4
107/107 [==============================] - 203s 2s/step - loss: 0.7530 - accuracy: 0.7915 - precision: 0.8696 - recall: 0.6991 - AUC: 0.9568
Epoch 2/4
107/107 [==============================] - 210s 2s/step - loss: 0.2451 - accuracy: 0.9311 - precision: 0.9514 - recall: 0.8987 - AUC: 0.9940
Epoch 3/4
107/107 [==============================] - 210s 2s/step - loss: 0.1062 - accuracy: 0.9723 - precision: 0.9754 - recall: 0.9588 - AUC: 0.9989
Epoch 4/4
107/107 [==============================] - 229s 2s/step - loss: 0.0481 - accuracy: 0.9906 - precision: 0.9929 - recall: 0.9870 - AUC: 0.9995


In [56]:
import numpy as np
import csv
from tqdm import tqdm

test_path = './test/test/0'

with open('test_answer.csv', 'w' , newline='') as f:
    wr = csv.writer(f)
    wr.writerow(['','answer value'])
    for i, image in tqdm(enumerate(os.listdir(test_path))) :
        img = cv2.imread(os.path.join(test_path, image))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, dsize=(227,227))
        img = tf.reshape(img, (-1, img.shape[0], img.shape[1], img.shape[2]))

        pred = model.predict(img)
        answer = np.argmax(pred[0])
        wr.writerow([i, answer])
    
    

350it [00:38,  9.02it/s]
